In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [3]:
train_data=pd.read_csv('../input/titanic/train.csv')
train_data

In [4]:
train_data=train_data.fillna(0)

In [5]:
train_data

In [6]:
from sklearn.preprocessing import LabelEncoder
cols=['Sex','Embarked']
le=LabelEncoder()

for col in cols:
    train_data[col] = le.fit_transform(train_data[col].astype(str))
train_data.head()

## Drop names

In [7]:
train_data=train_data.drop(['Name','Ticket'],axis=1)
train_data

In [8]:
selected_features=['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
X_train=train_data[selected_features]
#X_train=pd.get_dummies(X_train)
X_train

In [9]:
Y_train=train_data['Survived']
Y_train

## Correlation

In [10]:
corr = X_train.corr()
plt.figure(figsize=(15, 9))
sns.heatmap(corr, annot=True)

In [11]:
#pip install tensorflow

In [12]:
import tensorflow as tf
import keras
import numpy as np 
import pandas as pd
import os
from keras.models import Sequential
from keras import layers
from keras.layers import BatchNormalization, Convolution1D, MaxPooling1D, Convolution2D , MaxPooling2D
from keras.layers.core import  Reshape , Dense, Flatten, Dropout
from sklearn.model_selection import train_test_split

In [13]:
X_train=np.array(X_train)
Y_train=np.array(Y_train)

In [14]:
x_train,x_test, y_train, y_test = train_test_split(np.array(X_train), Y_train, test_size=0.3, shuffle=True) # test_size é o percentual do dataset que vai ser usado nas amostras
#x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, shuffle=True) # test_size é o percentual do dataset que vai ser usado nas amostras

In [15]:
#x_train=np.array(np.reshape(np.array(x_train),((np.array(x_train).shape[0],np.array(x_train).shape[1]))))
#x_test=np.array(np.reshape(np.array(x_test),((np.array(x_test).shape[0],np.array(x_test).shape[1]))))

In [16]:
print(x_train.shape,y_train.shape,x_test.shape,y_test.shape)

In [17]:
model = Sequential([
    keras.models.InputLayer(input_shape = (x_train.shape[1])),
    #Dense(512, activation='sigmoid'),
    #Dropout(0.3),
    #Dense(256, activation='relu'),
    #Dropout(0.2),
    Dense(128, activation='sigmoid'),
    Dropout(0.2),
    Dense(128, activation='sigmoid'),
    Dropout(0.2),
    Dense(128, activation='sigmoid'),
    Dropout(0.2),
    Dense(128, activation='sigmoid'),
    Dropout(0.1),
    Dense(16, activation='sigmoid'),
    Dropout(0.1),
    Dense(1, activation='sigmoid')
])

model.summary()

In [18]:
model.compile(optimizer=tf.keras.optimizers.RMSprop(learning_rate=1e-3),loss=keras.losses.binary_crossentropy,metrics=["accuracy"])

history = model.fit(x_train,y_train, epochs = 25,validation_data=(x_test, y_test), verbose =-1, shuffle=True, workers=-1)

score= model.evaluate(x_test,y_test,batch_size=32)
score

In [19]:
# summarize history for accuracy
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [20]:
test_labels=pd.read_csv('../input/titanic/test.csv')
test_labels=test_labels.fillna(0)
test_labels

In [21]:
from sklearn.preprocessing import LabelEncoder
cols=['Sex','Embarked']
le=LabelEncoder()

for col in cols:
    test_labels[col] = le.fit_transform(test_labels[col].astype(str))
test_labels.head()

In [22]:
test_labels=test_labels.drop(['Name','Ticket'],axis=1)
selected_features=['PassengerId','Pclass','Sex','Age','SibSp','Parch','Fare','Embarked']
test_labels=test_labels[selected_features]
#test_labels=pd.get_dummies(test_labels)
test_labels_processed=test_labels #np.array(test_labels)
test_labels_processed

In [23]:
y_pred=model.predict(test_labels_processed)
y_pred
predict = np.argmax(y_pred, axis=1)

indices = []
#for i in range(1,len(predict)+1):
#    indices.append(i)

data_contatenada = np.array((test_labels.PassengerId,predict))
data_contatenada

submissions = pd.DataFrame(data=data_contatenada.T, columns= ['PassengerId','Survived'])
submissions.to_csv('./submission.csv', index=False)
submissions
